In [1]:
import pandas as pd
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np
from scrape_data import run_scrapper


In [2]:
first_page = 'https://www.linkedin.com/jobs/search/?currentJobId=3433766873&f_JT=F&f_T=25206&geoId=101282230&keywords=machine%20learning%20engineer&location=Germany&refresh=true&sortBy=R'
df = run_scrapper(first_page)
df.head()

ConnectionError: HTTPSConnectionPool(host='www.linkedin.com', port=443): Max retries exceeded with url: /jobs/search/?currentJobId=3433766873&f_JT=F&f_T=25206&geoId=101282230&keywords=machine%20learning%20engineer&location=Germany&refresh=true&sortBy=R (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff7d9a4d0d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, model_outputs):
        results = super().postprocess(
            model_outputs=model_outputs,
            aggregation_strategy=AggregationStrategy.SIMPLE,
        )
        return np.unique([result.get("word").strip() for result in results])
# Load pipeline
model_name = "ml6team/keyphrase-extraction-kbir-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

In [ ]:
data_keyword = []
def get_keywords(text):
    return extractor(text)
# text_data = df['job_description'].to_list()
df['key_words'] = df.apply(lambda x:get_keywords(x.job_description), axis=1)
df.head()

In [ ]:
df['key_words'].tolist()